In [1]:
import numpy as np

In [4]:
def compute_dists(arr):
    n, d = arr.shape
    dist = np.ones(n,n,dtype=np.float64)
    for i in range(n):
        for j in range(i+1, n):
            d = np.sum(np.square(arr[i,:]-arr[j,:]))
            dist[i,j]=d
            dist[j,i]=d
    return dist

In [5]:
def get_k_closest(dist, k):
    partition = np.partition(dist,k, axis=1)
    return partition[:,:k]

In [74]:
from numba.typed import List
def forest(closest_neighbors):
    n = closest_neighbors.shape[0]
    k = closest_neighbors.shape[1]

    edges = np.empty((n,2), dtype=np.int64)
    current_tree = np.empty(n, dtype=np.int64)
    current_size = 0

    partition = np.full(n, -1, dtype=np.int64)
    components = List()
    num_edges = 0
    
    order = np.random.permutation(n)
    for v in order:
        if partition[v]>=0:
            continue
        current_tree[0] = v
        current_size = 1
        partition[v] = len(components)
        other_partition = -1
        while current_size < k:
            for neigh in closest_neighbors[v]: # this may be randomized as well
                if partition[neigh]==len(components):
                    continue
                edges[num_edges,0] = v
                edges[num_edges,1] = neigh
                num_edges+=1
                if partition[neigh]>=0:
                    other_partition = partition[neigh]
                    break
                current_tree[current_size]=neigh
                current_size+=1
                v = neigh
                partition[v]=len(components)
                
            if other_partition!=-1:
                break
        if other_partition == -1:
            components.append(current_tree[0:current_size].copy())
        else:
            for node in current_tree[0:current_size]:
                occupied = other_partition
    return partition, components, edges[:num_edges,:]

In [75]:
closest_neighbors = np.array([[1,2], [2,0], [1,0], [4,5],[5,3],[3,4]], dtype=np.int64)

In [76]:
np.random.seed(2)
forest(closest_neighbors)

(array([1, 1, 1, 0, 0, 0]),
 ListType[array(int64, 1d, C)]([[4 5 3], [1 2 0], ...]),
 array([[4, 5],
        [5, 3],
        [1, 2],
        [2, 0]]))

In [77]:
np.random.seed(1)
forest(closest_neighbors)

(array([0, 0, 0, 1, 1, 1]),
 ListType[array(int64, 1d, C)]([[2 1 0], [4 5 3], ...]),
 array([[2, 1],
        [1, 0],
        [4, 5],
        [5, 3]]))

In [ ]:
def decompose_components(components, edges):
    for component in components:


def split_component(component, degrees, k):
    if len(component) <= max(2*k-1, 3*k-5):
        l = List()
        l.append(component)
        return l
    for u in component:
        if degrees[u] == 1:
            continue
        
